In [1]:
import pandas as pd
from copy import deepcopy

In [2]:
# Read the data and find all cost types
allocation_data = pd.read_csv("attribution_allocation_student_data.csv")
allocation_data.fillna(0,inplace=True)
channel_spend = pd.read_csv("channel_spend_student_data.csv")
tiers_cost = list(channel_spend.iloc[:,1])
tiers_cost = [eval(tier_cost) for tier_cost in tiers_cost]
cost_types = list(tiers_cost[0].keys())

In [3]:
# Create the subset of different convert condition and tiers
convert_TF = [True,False]
tiers = [1,2,3]
allocation_group = [[[0,dict()] for _ in tiers] for _ in convert_TF]
for i,convert in enumerate(convert_TF):
    for j,tier in enumerate(tiers):
        allocation_group[i][j] = allocation_data[(allocation_data["convert_TF"]==convert) & (allocation_data["tier"]==tier)].iloc[:,1:6]

In [4]:
# Compute the credit of each subset and find the total credit for each channel
def count(x,cost,option):
    start = 0
    end = len(x)-1
    while x[end]==0:
        end-=1
    if option==1:
        cost[x[end]]+=1
    elif option==2:
        if end-start<=1:
            cost[x[start]]+=0.5
            cost[x[end]]+=0.5
        else:
            cost[x[start]]+=0.4
            cost[x[end]]+=0.4
            size = end-start-1
            for i in range(size):
                cost[x[start+i+1]]+=0.2/size
    elif option==3:
        size = end-start+1
        for i in range(size):
            cost[x[start+i]] += 1/size

cost = dict()
for i in cost_types:
    cost[i] = 0
number_cus = [[deepcopy(cost) for _ in tiers] for _ in convert_TF]
options = {"Last":1,"Position":2,"Linear":3}
option = options["Linear"]
for i,convert in enumerate(convert_TF):
    for j,tier in enumerate(tiers):
        allocation_group[i][j].apply(lambda x: count(x,number_cus[i][j],option),axis=1)

In [5]:
# Compute the CAC for each tier each channel (linear)
CACs = [[0 for _ in cost_types] for _ in tiers]
for i in range(len(tiers)):
    for j,cost_type in enumerate(cost_types):
        CACs[i][j] = tiers_cost[i][cost_type]/number_cus[0][i][cost_type]
CACs = pd.DataFrame(CACs,columns=cost_types)
CACs["tiers"] = [1,2,3]
CACs.set_index("tiers")

,email,social,display,paid_search,referral,organic_search,direct
tiers,,,,,,,
1,3.981949,2.063629,2.276521,7.809449,0.659152,0.0,0.0
2,4.519604,2.432005,2.762113,8.409840,0.752818,0.0,0.0
3,5.159515,3.014368,3.545610,8.849558,0.911840,0.0,0.0


In [7]:
# Compute the CAC for each tier each channel (last)
option = options["Last"]
for i,convert in enumerate(convert_TF):
    for j,tier in enumerate(tiers):
        allocation_group[i][j].apply(lambda x: count(x,number_cus[i][j],option),axis=1)
# Compute the CAC for each tier each channel
CACs = [[0 for _ in cost_types] for _ in tiers]
for i in range(len(tiers)):
    for j,cost_type in enumerate(cost_types):
        CACs[i][j] = tiers_cost[i][cost_type]/number_cus[0][i][cost_type]
CACs = pd.DataFrame(CACs,columns=cost_types)
CACs["tiers"] = [1,2,3]
CACs.set_index("tiers")

,email,social,display,paid_search,referral,organic_search,direct
tiers,,,,,,,
1,1.879228,0.901239,1.033841,3.288933,0.366555,0.0,0.0
2,2.004979,1.062492,1.296947,3.419875,0.419843,0.0,0.0
3,2.426301,1.321450,1.673998,3.658537,0.501165,0.0,0.0


In [8]:
# Compute the CAC for each tier each channel (position)
option = options["Position"]
for i,convert in enumerate(convert_TF):
    for j,tier in enumerate(tiers):
        allocation_group[i][j].apply(lambda x: count(x,number_cus[i][j],option),axis=1)
# Compute the CAC for each tier each channel
CACs = [[0 for _ in cost_types] for _ in tiers]
for i in range(len(tiers)):
    for j,cost_type in enumerate(cost_types):
        CACs[i][j] = tiers_cost[i][cost_type]/number_cus[0][i][cost_type]
CACs = pd.DataFrame(CACs,columns=cost_types)
CACs["tiers"] = [1,2,3]
CACs.set_index("tiers")

,email,social,display,paid_search,referral,organic_search,direct
tiers,,,,,,,
1,1.258917,0.613717,0.700215,2.247275,0.240090,0.0,0.0
2,1.370661,0.722282,0.873521,2.369996,0.274246,0.0,0.0
3,1.625767,0.898858,1.123224,2.514880,0.329267,0.0,0.0
